In [65]:
from sympy import symbols, Matrix, diff, cos, sin, pi, diff,Eq,sqrt
from sympy import Rational as rat
import sympy as sp
from sympy.physics.vector import dynamicsymbols
t,z,h,m,mf,mc,d,g,j,s,Kp_theta,Kd_theta,Kp_z,Kd_Z,K_DC,b = symbols('t,z,h,m,m_f,m_c,d,g,j,s,Kp_theta,Kd_theta,Kp_z,Kd_Z,K_DC,b')


In [109]:
z_d, h_d, theta_d, Fl_d, Fr_d, F_d, T_d = dynamicsymbols('z, h, theta, F_l, F_r, F, T', 0)
zD_d, hD_d, thetaD_d = dynamicsymbols('z, h, theta', 1)
zDD_d, hDD_d, thetaDD_d = dynamicsymbols('z, h, theta', 2)
Dynamic = [zDD_d, hDD_d, thetaDD_d, zD_d, hD_d, thetaD_d, z_d, h_d, theta_d, F_l, F_r, F_d]

z_s, h_s, theta_s, Fl_s, Fr_s, F_s, T_s = symbols('z, h, theta, Fl, Fr, F, T')
zD_s, hD_s, thetaD_s = symbols('zDot, hDot, thetaDot')
zDD_s, hDD_s, thetaDD_s = symbols('zDDot, hDDot, thetaDDot')
Static = [zDD_s, hDD_s, thetaDD_s, zD_s, hD_s, thetaD_s, z_s, h_s, theta_s, F_s, T_s]

zTilde, hTilde, thetaTilde, zDTilde, hDTilde, thetaDTilde, zDDTilde, hDDTilde, thetaDDTilde, FTilde, TTilde = symbols('zTilde, hTilde, thetaTilde, zDotTilde, hDotTilde, thetaDotTilde, zDDotTilde, hDDotTilde, thetaDDotTilde, FTilde, TTilde')

Pl = Matrix([[z_d - d * cos(theta_d), h_d - d * sin(theta_d), 0]]).T
Pr = Matrix([[z_d + d * cos(theta_d), h_d + d * sin(theta_d), 0]]).T
Pc = Matrix([[z_d, h_d, 0]]).T

Vl = sp.diff(Pl, t)
Vr = sp.diff(Pr, t)
Vc = sp.diff(Pc, t)

K = (rat(1,2) * mf * Vl.T * Vl)[0] + (rat(1,2) * mf * Vr.T @ Vr)[0] + (rat(1,2) * mc * Vc.T @ Vc)[0]

# Kinetic Energy
K = K.subs(zip((zD_d, hD_d, thetaD_d), (zD_s, hD_s, thetaD_s))).simplify().factor(zD_s, thetaD_s, hD_s).subs(zip((zD_s, hD_s, thetaD_s), (zD_d, hD_d, thetaD_d)))

# Potential Energy
P = g * h_d * (2 * mf + mc)

# Lagrangian
L = K - P

# Euler Lagrange equations
EL = Matrix([[sp.diff(sp.diff(L, zD_d), t) - sp.diff(L, z_d)],
             [sp.diff(sp.diff(L, hD_d), t) - sp.diff(L, h_d)],
             [sp.diff(sp.diff(L, thetaD_d), t) - sp.diff(L, theta_d)]])


# Applied Force
tau = Matrix([[(Fl_d + Fr_d) * sin(-theta_d)],
              [(Fl_d + Fr_d) * cos(theta_d)],
              [Fl_d * d - Fr_d * d]])

# Damping
Bq = Matrix([[b * zD_d],
              [b * hD_d],
              [b * thetaD_d]])

# Equations of motion
EOM = Eq(EL, tau - Bq)


EOM = EOM.subs(zip((zDD_d, hDD_d, thetaDD_d, zD_d, hD_d, thetaD_d, z_d, h_d, theta_d, Fl_d, Fr_d),
                   (zDD_s, hDD_s, thetaDD_s, zD_s, hD_s, thetaD_s, z_s, h_s, theta_s, Fl_s, Fr_s)))
S = sp.solve(EOM, [zDD_s, hDD_s, thetaDD_s])
EOM = Matrix([[S[zDD_s].simplify().factor(sin(theta_s)).subs(Fl_s+Fr_s, F_s)],
              [S[hDD_s].simplify().factor(cos(theta_s), g).subs(Fl_s+Fr_s, F_s)],
              [S[thetaDD_s].simplify().factor(d).subs(d * (Fl_s - Fr_s), T_s)]])

# Equilibrium conditions
temp = Eq(Matrix([[0, 0, 0]]).T, EOM).subs([(zD_s, 0), (hD_s, 0), (thetaD_s, 0), (theta_s, 0)])

S = sp.solve(temp, [F_s, T_s])
Fe = S[F_s]
Te = S[T_s]
theta_e = 0
zDD_e = 0
hDD_e = 0
thetaDD_e = 0
zD_e = 0
hD_e = 0
thetaD_e = 0

EOM = Matrix([[zD_s,hD_s,thetaD_s]]).T.col_join(EOM)
# Jacobian state space linearization
state_vars = Matrix([z_s, h_s, theta_s, zD_s, hD_s, thetaD_s])
input_vars = Matrix([F_s, T_s])
A = EOM.jacobian(state_vars)
B = EOM.jacobian(input_vars)
display(A)
equilibrium = {
    theta_s: 0,
    F_s: g * (mc + 2 * mf),
    zD_s: 0,
    hD_s: 0,
    thetaD_s: 0,
    T_s: 0
}
A = A.subs(equilibrium)
B = B.subs(equilibrium)
C = Matrix([[1,0,0],
            [0,1,0],
            [0,0,1]])

x_tilde = Matrix([[zTilde, hTilde, thetaTilde, zDTilde, hDTilde, thetaDTilde]]).T
u_tilde = Matrix([[FTilde, TTilde]]).T
xDot_tilde = Matrix([[zDTilde, hDTilde, thetaDTilde, zDDTilde, hDDTilde, thetaDDTilde]]).T

# State-space model
print("Equations of Motion:")
display(EOM)
print("The linear state space:")
display(Eq(xDot_tilde, sp.Add(sp.MatMul(A, x_tilde), sp.MatMul(B, u_tilde)), evaluate=False))

# Linearized EOMs
LIN_EOM = Eq(Matrix([[zDDTilde], [hDDTilde], [thetaDDTilde]]), A @ x_tilde - B @ u_tilde)
print("Simplified linear equations of motion")
display(A @ x_tilde - B @ u_tilde)


Matrix([
[0, 0,                           0,                1,                0,               0],
[0, 0,                           0,                0,                1,               0],
[0, 0,                           0,                0,                0,               1],
[0, 0, -F*cos(theta)/(m_c + 2*m_f), -b/(m_c + 2*m_f),                0,               0],
[0, 0, -F*sin(theta)/(m_c + 2*m_f),                0, -b/(m_c + 2*m_f),               0],
[0, 0,                           0,                0,                0, -b/(2*d**2*m_f)]])

Equations of Motion:


Matrix([
[                                                    zDot],
[                                                    hDot],
[                                                thetaDot],
[                  -(F*sin(theta) + b*zDot)/(m_c + 2*m_f)],
[(F*cos(theta) - b*hDot + g*(-m_c - 2*m_f))/(m_c + 2*m_f)],
[                           (T - b*thetaDot)/(2*d**2*m_f)]])

The linear state space:


Eq(Matrix([
[     zDotTilde],
[     hDotTilde],
[ thetaDotTilde],
[    zDDotTilde],
[    hDDotTilde],
[thetaDDotTilde]]), Matrix([
[              0,              0],
[              0,              0],
[              0,              0],
[              0,              0],
[1/(m_c + 2*m_f),              0],
[              0, 1/(2*d**2*m_f)]])*Matrix([
[FTilde],
[TTilde]]) + Matrix([
[0, 0,  0,                1,                0,               0],
[0, 0,  0,                0,                1,               0],
[0, 0,  0,                0,                0,               1],
[0, 0, -g, -b/(m_c + 2*m_f),                0,               0],
[0, 0,  0,                0, -b/(m_c + 2*m_f),               0],
[0, 0,  0,                0,                0, -b/(2*d**2*m_f)]])*Matrix([
[       zTilde],
[       hTilde],
[   thetaTilde],
[    zDotTilde],
[    hDotTilde],
[thetaDotTilde]]))

Simplified linear equations of motion


Matrix([
[                                          zDotTilde],
[                                          hDotTilde],
[                                      thetaDotTilde],
[          -b*zDotTilde/(m_c + 2*m_f) - g*thetaTilde],
[  -FTilde/(m_c + 2*m_f) - b*hDotTilde/(m_c + 2*m_f)],
[-TTilde/(2*d**2*m_f) - b*thetaDotTilde/(2*d**2*m_f)]])

FULL-STATE FEEDBACK

In [105]:
#Controllability Matrix
import numpy as np
from scipy.linalg import solve_continuous_are
from scipy.signal import place_poles

#break into lat and long
A_lat = Matrix([[0,0,1,0],
                [0,0,0,1],
                [0,-g,-b/(mc+2*mf),0],
                [0,0,0,-b/(2*d**2*mf)]])
B_lat = Matrix([[0],
                [0],
                [0],
                [1/(2*d**2*mf)]])
xDot_lat = Matrix([[zDTilde],
                   [thetaDTilde],
                   [zDDTilde],
                   [thetaDDTilde]])
x_lat = Matrix([[z_s,theta_s,zD_s,thetaD_s]]).T
u_lat = Matrix([[T_s]])
display(Eq(xDot_lat, sp.Add(sp.MatMul(A_lat, x_lat), sp.MatMul(B_lat, u_lat)), evaluate=False))


A_long = Matrix([[0,1],
                 [0,-b/(mc+2*mf)]])
B_long = Matrix([[0],
                 [1/(mc*2*mf)]])
xDot_long = Matrix([[hDTilde,hDDTilde]]).T
x_long = Matrix([[h_s,hD_s]]).T
u_long = Matrix([[F_s]])
display(Eq(xDot_long, sp.Add(sp.MatMul(A_long, x_long), sp.MatMul(B_long, u_long)), evaluate=False))


Eq(Matrix([
[     zDotTilde],
[ thetaDotTilde],
[    zDDotTilde],
[thetaDDotTilde]]), Matrix([
[             0],
[             0],
[             0],
[1/(2*d**2*m_f)]])*Matrix([[T]]) + Matrix([
[0,  0,                1,               0],
[0,  0,                0,               1],
[0, -g, -b/(m_c + 2*m_f),               0],
[0,  0,                0, -b/(2*d**2*m_f)]])*Matrix([
[       z],
[   theta],
[    zDot],
[thetaDot]]))

Eq(Matrix([
[ hDotTilde],
[hDDotTilde]]), Matrix([
[            0],
[1/(2*m_c*m_f)]])*Matrix([[F]]) + Matrix([
[0,                1],
[0, -b/(m_c + 2*m_f)]])*Matrix([
[   h],
[hDot]]))

In [2]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
from sympy import cos,sin
theta,z,h=dynamicsymbols('theta,z,h')
thetaDot,zDot,hDot=dynamicsymbols('theta,z,h',1)
thetaDDot,zDDot,hDDot=dynamicsymbols('theta,z,h',2)
b,Fl,Fr,g,L,t,mf,mc,m,Theta,Z,H,ThetaDot,ZDot,HDot,ThetaDDot,ZDDot,HDDot = sp.symbols('b,F_l,F_r,g,L,t,m_f,m_c,m,theta,z,h,thetaDot,zDot,hDot,thetaDDot,zDDot,hDDot')


Pl = sp.Matrix([[z-L*cos(theta), h-L*sin(theta), 0]]).T
Pr = sp.Matrix([[z+L*cos(theta), h+L*sin(theta), 0]]).T
Pc = sp.Matrix([[z,h,0]]).T

Vl = sp.diff(Pl,t)
Vr = sp.diff(Pr,t)
Vc = sp.diff(Pc,t)

display(Vl)
display(Vr)
display(Vc)

Matrix([
[ L*sin(theta(t))*Derivative(theta(t), t) + Derivative(z(t), t)],
[-L*cos(theta(t))*Derivative(theta(t), t) + Derivative(h(t), t)],
[                                                             0]])

Matrix([
[-L*sin(theta(t))*Derivative(theta(t), t) + Derivative(z(t), t)],
[ L*cos(theta(t))*Derivative(theta(t), t) + Derivative(h(t), t)],
[                                                             0]])

Matrix([
[Derivative(z(t), t)],
[Derivative(h(t), t)],
[                  0]])

In [3]:
print('position center')
display(Pc)

print('\nposition left')
display(Pl)

print('\nposition right')
display(Pr)

print('\nvelocity center')
display(Vc)

print('\nvelocity left')
display(Vl)

print('\nvelcotiy right')
display(Vr)


T1 = ((Vl.T@Vl)[0].subs([(thetaDot,ThetaDot),(zDot,ZDot),(hDot,HDot), (theta,Theta),(h,H),(z,Z)]).expand().simplify())
T2 = ((Vr.T@Vr)[0].subs([(thetaDot,ThetaDot),(zDot,ZDot),(hDot,HDot), (theta,Theta),(h,H),(z,Z)]).expand().simplify())
T3 = ((Vc.T@Vc)[0].subs([(thetaDot,ThetaDot),(zDot,ZDot),(hDot,HDot), (theta,Theta),(h,H),(z,Z)]).expand().simplify())

print("\nVl.T@Vl")
display(T1)

print("\nVr.T@Vr")
display(T2)

print("\nVc.T@Vc")
display(T3)

K = (1/2*mf*T1 + 1/2*mf*T2 + 1/2*mc*T3).factor(zDot,thetaDot,hDot).simplify()
print('\nKinetic Energy')
display(K)

P = m*g*h
print('potential energy')
display(P)

Lag = K-P
print('Lagrangian')
display(Lag)
Lag = Lag.subs([(ThetaDot,thetaDot),(ZDot,zDot),(HDot,hDot), (Theta,theta),(H,h),(Z,z)])

position center


Matrix([
[z(t)],
[h(t)],
[   0]])


position left


Matrix([
[-L*cos(theta(t)) + z(t)],
[-L*sin(theta(t)) + h(t)],
[                      0]])


position right


...

1.0*L**2*m_f*thetaDot**2 + 0.5*hDot**2*m_c + 1.0*hDot**2*m_f + 0.5*m_c*zDot**2 + 1.0*m_f*zDot**2

potential energy


g*m*h(t)

Lagrangian


1.0*L**2*m_f*thetaDot**2 - g*m*h(t) + 0.5*hDot**2*m_c + 1.0*hDot**2*m_f + 0.5*m_c*zDot**2 + 1.0*m_f*zDot**2

In [4]:
pL_pqDot = sp.Matrix([[sp.diff(Lag,zDot).simplify(),sp.diff(Lag,hDot),sp.diff(Lag,thetaDot).simplify()]]).T

print('derivative of lagrangian with respect to velocity:')
display(pL_pqDot.subs([(thetaDot,ThetaDot),(zDot,ZDot),(hDot,HDot), (theta,Theta),(h,H),(z,Z)]))


T1 = sp.diff(pL_pqDot,t)
print("first term of Euler Lagrange")
display(T1.subs([(thetaDDot,ThetaDDot),(zDDot,ZDDot),(hDDot,HDDot),(thetaDot,ThetaDot),(zDot,ZDot),(hDot,HDot), (theta,Theta),(h,H),(z,Z)]))


T2 = sp.Matrix([[sp.diff(Lag,z), sp.diff(Lag,h), sp.diff(Lag,theta)]]).T
print("second term of Euler Lagrange")
display(T2)

tau = sp.Matrix([[-(Fl+Fr)*sin(theta), (Fl+Fr)*cos(theta), L*Fl-L*Fr]]).T
print("applied force")
display(tau)

qDot = sp.Matrix([[zDot,hDot,thetaDot]]).T
EOM = sp.Eq(T1-T2,tau-b*qDot)
print("equations of motion for planar Vtol")
display(EOM.subs([(thetaDDot,ThetaDDot),(zDDot,ZDDot),(hDDot,HDDot),(thetaDot,ThetaDot),(zDot,ZDot),(hDot,HDot), (theta,Theta),(h,H),(z,Z)]))

S = sp.solve(EOM,[zDDot,hDDot,thetaDDot])
EOM = sp.Matrix([[S[zDDot].simplify(), S[hDDot].simplify(), S[thetaDDot].simplify()]]).T
print('Solved equations of motion')
display(EOM.subs([(thetaDDot,ThetaDDot),(zDDot,ZDDot),(hDDot,HDDot),(thetaDot,ThetaDot),(zDot,ZDot),(hDot,HDot), (theta,Theta),(h,H),(z,Z)]))

derivative of lagrangian with respect to velocity:


Matrix([
[   zDot*(1.0*m_c + 2.0*m_f)],
[1.0*hDot*m_c + 2.0*hDot*m_f],
[      2.0*L**2*m_f*thetaDot]])

first term of Euler Lagrange


Matrix([
[    zDDot*(1.0*m_c + 2.0*m_f)],
[1.0*hDDot*m_c + 2.0*hDDot*m_f],
[       2.0*L**2*m_f*thetaDDot]])

second term of Euler Lagrange


...

Matrix([
[(-F_l - F_r)*sin(theta(t))],
[ (F_l + F_r)*cos(theta(t))],
[             F_l*L - F_r*L]])

equations of motion for planar Vtol


Eq(Matrix([
[          zDDot*(1.0*m_c + 2.0*m_f)],
[g*m + 1.0*hDDot*m_c + 2.0*hDDot*m_f],
[             2.0*L**2*m_f*thetaDDot]]), Matrix([
[-b*zDot + (-F_l - F_r)*sin(theta)],
[ -b*hDot + (F_l + F_r)*cos(theta)],
[       F_l*L - F_r*L - b*thetaDot]]))

Solved equations of motion


Matrix([
[     -(F_l*sin(theta) + F_r*sin(theta) + b*zDot)/(m_c + 2.0*m_f)],
[(F_l*cos(theta) + F_r*cos(theta) - b*hDot - g*m)/(m_c + 2.0*m_f)],
[                     0.5*(F_l*L - F_r*L - b*thetaDot)/(L**2*m_f)]])

In [5]:
ztDDot,htDDot,thetatDDot,ztDot,htDot,thetatDot,zt,ht,thetat,Ft,Tt,s = sp.symbols('ztildeDDot,htildeDDot,thetatildeDDot,ztildeDot,htildeDot,thetatildeDot,ztilde,htilde,thetatilde,Ftilde,Ttilde,s')
Z,H,Theta,F,T = [i.subs(t,s) for i in dynamicsymbols('Ztilde,Htilde,Thetatilde,Ftilde,Ttilde')]


LEOM = sp.Matrix([[-thetat*g-b*ztDot/m],
                  [Ft/m-b*htDot/m],
                  [Tt/(2*L**2*mf)-thetatDot*b/(2*L**2*mf)]])
print('linearized equations of motion')
display(LEOM)

angle_TO_alt = sp.Eq(Z,(1/m)/(s**2+b/m*s) * Theta)
Force_TO_dist = sp.Eq(H,(1/m)/(s**2+b/m*s) * F)
Torque_TO_angle = sp.Eq(Theta,1/(s**2+s*b/(2*L**2*mf) * 1/(2*L**2*mf)) * T)

print("Laplace Domani Equations")
display(angle_TO_alt)
display(Force_TO_dist)
display(Torque_TO_angle)

print("transfer function of torque to altitude")
Torque_TO_alt = angle_TO_alt.subs(Theta,Torque_TO_angle.rhs).simplify()
display(Torque_TO_alt)

linearized equations of motion


Matrix([
[                     -b*ztildeDot/m - g*thetatilde],
[                          Ftilde/m - b*htildeDot/m],
[Ttilde/(2*L**2*m_f) - b*thetatildeDot/(2*L**2*m_f)]])

Laplace Domani Equations


Eq(Ztilde(s), Thetatilde(s)/(m*(b*s/m + s**2)))

Eq(Htilde(s), Ftilde(s)/(m*(b*s/m + s**2)))

Eq(Thetatilde(s), Ttilde(s)/(s**2 + b*s/(4*L**4*m_f**2)))

transfer function of torque to altitude


Eq(Ztilde(s), 4*L**4*m_f**2*Ttilde(s)/(s**2*(b + m*s)*(4*L**4*m_f**2*s + b)))